In [236]:
import theano
import numpy as np
import theano.tensor as T

from theano.ifelse import ifelse
from theano.printing import Print

In [301]:
x = np.array([1, 0, -1])

def conditional_sum():
    
    x = T.fvector('x')
    
    s = ifelse(T.gt(T.sum(T.eq(x,0)), 0), Print('Sum(x)')(T.sum(T.eq(x,0), dtype='float32')), T.sum(x) + 1)
    
    return theano.function([x], s, allow_input_downcast=True)

f = conditional_sum()

f(x)

Sum(x) __str__ = 1.0


array(1.0, dtype=float32)

In [296]:
x = np.random.rand(10, 8)
i = np.array([4,3,1,6,0,2,2,0,7,1])
i_one_hot = np.eye(10, dtype=np.bool)[i]

def indexing():
    
    x = T.fmatrix('x')
    i = T.ivector('i')
    
    res = x[T.arange(10), i]
    
    return theano.function([x, i], res, allow_input_downcast=True)

f = indexing()

f(x, i)

array([ 0.30282405,  0.12803105,  0.76441413,  0.15160133,  0.41486189,
        0.81014425,  0.79314798,  0.83336681,  0.27174285,  0.77675211], dtype=float32)

In [297]:
x[np.arange(len(x)), i]

array([ 0.30282405,  0.12803105,  0.76441415,  0.15160133,  0.41486188,
        0.81014427,  0.79314795,  0.83336681,  0.27174284,  0.77675212])

In [266]:
i_one_hot

array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [140]:
x = np.array([[1,2,3,4,9,9,9], [5,6,7,3,2,1,9]])

def eos_mask():
    
    x = theano.gradient.disconnected_grad(T.imatrix('x'))
    
    mask = T.switch(T.eq(x, 9), T.arange(0, 7), 10)
#     eos_index = mask[mask > 0]
    eos_index = T.tile(mask.min(axis=-1), (7, 1)).T
#     test = T.tile(T.arange(0, 7), (x.shape[0], 1))
    binary_mask = T.where(T.ge(T.tile(T.arange(0, 7), (x.shape[0], 1)), eos_index), 0, 1)
#     first_eos_index = eos_index[0] - 1 
#     binary_mask = T.set_subtensor(mask[first_eos_index:], 1)
    masked_sum = T.sum(T.switch(T.eq(binary_mask, 1), 1, 0) * x, axis=-1)
    
    return theano.function([x], [eos_index, masked_sum, T.grad(T.sum(masked_sum), x)], allow_input_downcast=True)

f = eos_mask()

f(x)

[array([[4, 4, 4, 4, 4, 4, 4],
        [6, 6, 6, 6, 6, 6, 6]]),
 array([10, 24]),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)]

In [234]:
x = np.random.rand(4, 1, 3)
m = np.random.randint(0, size=(4, 1, 3), dtype=np.int64, high=2)

def bool_mask():
    
    x = T.ftensor3('x')
    m = T.ltensor3('m')
    
    x = Print('x', ['shape'])(T.squeeze(x))
    m = T.squeeze(x)
    res = x[T.nonzero(m)]
    
    return theano.function(inputs=[x, m], outputs=res, allow_input_downcast=True)

g = bool_mask()

g(x, m)

array([ 0.08683389,  0.77492368,  0.36511773,  0.73312443,  0.37144971], dtype=float32)

In [216]:
x

array([[ 0.02847295,  0.48462557,  0.66341043],
       [ 0.38974284,  0.55709656,  0.63985614],
       [ 0.56891956,  0.29013549,  0.49927048],
       [ 0.77266806,  0.3749504 ,  0.44598111]])

In [186]:
x = np.array([[0,1,3,5,6,-1], [0,5,6,7,8,-1], [4,2,1,4,5,-1]])
most_common = [1,2,5]
vocab_size = x.max()

In [17]:
from collections import Counter
c = Counter([1,2,4,4])

In [112]:
x = np.array([0,1,3,5,6,-1])
most_common = [1,2,5]
most_common = None
x[(x >= 0) & (~np.in1d(x, most_common))]

array([0, 1, 3, 5, 6])

In [174]:
s = set()
s.add(3)
set(s)

{3}

In [165]:
def frequency_vec(x, exclude, max_index):
    
    def array_to_list(array):
        c = Counter(array[(array >= 0) & (~np.in1d(array, exclude))])
        l = np.zeros((max_index + 1))
        l[list(c.keys())] = list(c.values())
        return l

    khot = np.array(list(map(lambda row: array_to_list(row), x)))
    if exclude is None:
        return khot
    else:
        return np.delete(arr=khot, axis=1, obj=exclude)
    
def theano_frequency_vec():
    
    X = T.matrix('X')   
    ret = frequency_vec(x, most_common, vocab_size)
    
    return theano.function(inputs=[x], outputs=ret, allow_input_downcast=True)

In [141]:
frequency_vec(x, most_common, vocab_size)

array([[ 0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  2.,  0.,  0.,  0.]])

In [185]:
most_common = set(most_common).add(3)
print(most_common)

None


In [209]:
a = [1,4,5]
a.append(6)
a

[1, 4, 5, 6]

In [203]:
most_common = [6]
frequency_vec(x, most_common, vocab_size)

array([[ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  1.,  1.,  0.,  2.,  1.,  0.,  0.]])

In [22]:
def func(x, a=3, b=3):
    return (x+a)**b

In [23]:
x = T.scalar('x')
out = func(x)
f = theano.function([x], out)

In [93]:
def get_func(*args):
    a = args[0]
    b = args[1]
    
    x = T.scalar('x')
    out = func(x, a, b)
    return theano.function(inputs=[x], outputs=out)

In [94]:
f = get_func(3,4)

In [95]:
f(4)

array(2401.0, dtype=float32)

In [98]:
def get_another_func(*args):
    f1 = args[0]
    
    z = T.scalar('z')
    out = np.sqrt(z + f1)
    return theano.function(inputs=[z], outputs=out, on_unused_input='ignore')

In [101]:
f2 = get_another_func(f(4))

In [102]:
f2(2)

array(49.02040481567383, dtype=float32)

In [35]:
np.sqrt(2+(4+3)**4)

49.020403915104573

In [36]:
def get_another_func_var(*args):
    f = args[0]
    x = args[1]
    
    f1 = f(x)
    z = T.scalar('z')
    out = np.sqrt(z + f1)
    return theano.function(inputs=[z], outputs=out)

In [39]:
f3 = get_another_func_var(f, 4)

In [42]:
f3(2)

array(49.02040481567383, dtype=float32)

In [64]:
def g1():    
    x = T.vector('x')
    y = T.scalar('y')
    z = x * T.shape_padright(y)
    return theano.function([x,y], z, allow_input_downcast=True)

In [86]:
f = g1()

In [67]:
g(np.array([3,4,5]), 2)

array([  6.,   8.,  10.], dtype=float32)

In [87]:
def g2():
    z = T.scalar('z')
    w = T.scalar('w')
    j = f(z) ** w
    return theano.function([z,w], j, allow_input_downcast=True)

In [22]:
import theano
import theano.tensor as T

import numpy as np

from lasagne.layers import InputLayer, LSTMLayer, RecurrentLayer, BiasLayer, NonlinearityLayer, ConcatLayer, \
ElemwiseMergeLayer, get_output
from lasagne.nonlinearities import sigmoid, softmax, tanh

l_in = InputLayer((None, 40, 300))

l_current = l_in

for h in range(4):

    l_current = LSTMLayer(l_current, num_units=500, nonlinearity=tanh)

l_out = RecurrentLayer(l_current, num_units=300, W_hid_to_hid=T.zeros, nonlinearity=None)

l_in_gate = InputLayer((None, 40, 300))

l_bias = BiasLayer(l_out)
l_sigm = NonlinearityLayer(l_bias, nonlinearity=sigmoid)

def gate():
    x = T.dtensor3('x')
    y = T.dtensor3('y')
    gate = (1 - x) * y
    
    return theano.function([x, y], gate, allow_input_downcast=True)

gate_fn = gate()

l_gate = ElemwiseMergeLayer([l_sigm, l_in_gate], merge_function=gate_fn)

l_concat = ConcatLayer([l_out, l_gate], axis=0)
l_final = BiasLayer(l_concat)

In [23]:
data = np.random.randn(40, 300)
fake_data = np.random.randn(30, 300)

In [24]:
get_output(l_final, data)

ValueError: get_output() was called with a single input expression on a network with multiple input layers. Please call it with a dictionary of input expressions instead.

In [7]:
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import theano
import theano.tensor as T
import numpy as np
x = T.imatrix()
l_in = InputLayer((3, ))
W = np.arange(3*5).reshape((3, 5)).astype('float32')
l1 = EmbeddingLayer(l_in, input_size=3, output_size=5, W=W)
output = get_output(l1, x)
f = theano.function([x], output)
x_test = np.array([[0, 2], [1, 2]]).astype('int32')
print(f(x_test).shape)
f(x_test)

(2, 2, 5)


array([[[  0.,   1.,   2.,   3.,   4.],
        [ 10.,  11.,  12.,  13.,  14.]],

       [[  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.]]], dtype=float32)